In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import chromedriver_autoinstaller
import pandas as pd
import re

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')

import spacy
nlp = spacy.load("en_core_web_sm")




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Emre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Emre\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [28]:

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

game_id = "413150"
steam_url = f"https://steamcommunity.com/app/{game_id}/reviews/?filterLanguage=english&p=1&browsefilter=toprated"
driver.get(steam_url)
driver.maximize_window()
time.sleep(5) #yükleme için bekleme
start_time = time.time() 


In [29]:
def scroll_the_website():

    while True:   

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        if time.time()-start_time > 200:
            break

scroll_the_website()

reviews = driver.find_elements(By.CLASS_NAME, 'apphub_CardTextContent')
labels = labels = driver.find_elements(By.CSS_SELECTOR, 'div.title')


## Datayı toplama


In [30]:
labels_list = []
review_list = []

def store_the_reviews():

    for review in reviews:
        review_text = review.text
        review_list.append(review_text)

def store_the_labels():

    for label in labels:
        labels_text = label.text
        labels_list.append(labels_text)

store_the_reviews()
store_the_labels()
driver.quit()

## Cleaning the data


In [31]:
cleaned_reviews = []  

for review_text in review_list:
    review_text = review_text.split("\n")[1:]
    review_text = [item.replace("Ürün ücretsiz alındı", "") for item in review_text]
    review_text = ' '.join(review_text)
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    review_text = review_text.lower()
    doc = nlp(review_text)
    review_text = " ".join([token.lemma_ for token in doc]) 
    cleaned_reviews.append(review_text)




In [32]:
df_data = pd.DataFrame({'review': cleaned_reviews, 'labels': labels_list})
df_data['labels'] = df_data['labels'].map({'Tavsiye Ediliyor': 1, 'Tavsiye Edilmiyor': 0})
df_data.head(len(df_data))
df_data.to_csv('preprocessed_data2.csv', index=False)